In [1]:
import psycopg2
import osgeo.ogr
import shapely
import geopandas as gpd

In [60]:
connection = psycopg2.connect(database="geotech_ocean_haz", user="postgres", password = "postgres")
cursor = connection.cursor()

In [ ]:
#path to coastline  of hawaii 
input_shape_coastline = "https://opendata.arcgis.com/datasets/045b1d5147634e2380566668e04094c6_3.geojson"

In [93]:
#select shark attacks   
cursor.execute(
'''SELECT 's.Date', 's.geometry'
FROM shark_attacks as s
WHERE st_intersects(s.geometry, 
					 st_transform(
						 st_buffer(
							 st_transform(
								 st_setsrid(
									 st_makepoint(-156.767903, 20.749255),4326),26904),20000),4326))'''
				   )
connection.commit()

In [94]:
rows_list=[]
for Date, geometry in cursor:
    data={'Date':Date,'geometry':geometry}
    rows_list.append(data)
#print(rows_list)
gdf=gpd.GeoDataFrame(rows_list).set_index('Date')
gdf.head()

,geometry
Date,
s.Date,s.geometry
s.Date,s.geometry
s.Date,s.geometry


In [89]:
#select bottom type
cursor.execute(
'''SELECT b.objectid, b.seabed, b.geometry 
FROM bottom_type as b
WHERE st_intersects(b.geometry, 
					 st_transform(
						 st_buffer(
							 st_transform(
								 st_setsrid(
									 st_makepoint(-156.076333, 20.815275),4326),26904),10000),4326)
				   )'''
)
connection.commit()

In [90]:
bottom_list=[]
for objectid, seabed, geometry in cursor:
    data={'OBJECTID':objectid,'SEABED':seabed,'geometry':geometry}
    bottom_list.append(data)


In [92]:
#print(bottom_list)
gdf=gpd.GeoDataFrame(bottom_list).set_index('OBJECTID')
gdf.head()

,SEABED,geometry
OBJECTID,,
4173,Mud,0101000020E6100000705083AD698163C0A728A43C1AD7...
4174,Mud,0101000020E61000008E69AB94358063C03F28FA2042D2...
